# Soil Freeze-Thaw support

The following notebook contains a walkthrough of the support for the Soil Freeze-Thaw (SFT) module

The notebook is organized in the following sections:
1. Icefabric API support
2. Icefabric CLI support

In [ ]:
# Running imports
import json
import os
import threading
import zipfile
from pathlib import Path

import httpx

# Changes the current working dir to be the project root
current_working_dir = Path.cwd()
os.chdir(Path.cwd() / "../../")
print(
    f"Changed current working dir from {current_working_dir} to: {Path.cwd()}. This must run at the project root"
)

In [ ]:
# Starting the API locally
def run_api():
    """Starts the icefabric API locally"""
    !python -m app.main --catalog sql


threading.Thread(target=run_api).start()

This will start the API on localhost:8000. This can be visited at: http://localhost:8000/docs 

![Icefabric API](../../docs/img/icefabric_api.png)

To test the SFT endpoint, we can use an example API call. This will return all config entries in JSON form. Each item in the output is the BMI Config contents for a specific catchment upstream of USGS 01010000

In [ ]:
# Make the request
response = httpx.get(
    "http://0.0.0.0:8000/v1/modules/sft/",
    params={
        "identifier": "01010000",  # the Gauge ID we're testing
        "domain": "conus_hf",  # The CONUS domain
        "use_schaake": "false",  # Specifying we're not using Schaake for the ice fraction setting
    },
    timeout=30.0,  # GLUE API requests can be slow depending on the network speed. Adding a 30s timeout to ensure requests go through
)

print(f"Status code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

### CLI

Now that we can return to the user all of the information / IPEs for a module, we can use the icefabric CLI to generate config files to disk for the user


In [ ]:
Path("/tmp/sft_output_test").mkdir(exist_ok=True, parents=True)

# Ensure the current working dir is where your `.pyiceberg.yaml` file is located
!uv run icefabric params --gauge 01010000 --module sft --domain conus --catalog sql --output /tmp/sft_output_test

Now that these are created, let's view the contents

In [ ]:
with zipfile.ZipFile("/tmp/sft_output_test/configs.zip", "r") as f:
    print("Files in archive:")
    for file_info in f.filelist:
        print(f"  {file_info.filename} ({file_info.file_size} bytes)")
    f.extractall("/tmp/sft_output_test/configs")

Let's view one of their contents

In [ ]:
files = Path("/tmp/sft_output_test/configs").glob("*")
first_file = list(files)[0]
content = first_file.read_text()
print(content)

We can also view the metadata.json file that was created. This will contain additional information about the query parameters used to make these configs

In [ ]:
metadata = Path("/tmp/sft_output_test/configs/metadata.json")
content = json.loads(metadata.read_text())

print(content)

Through either the API, or CLI, any modeler can create a BMI Config file for SFT that is compatible with NextGen